# Đánh giá Mô hình Hệ Gợi ý Hybrid

## Mục tiêu
Đánh giá hiệu suất của các mô hình:
- Collaborative Filtering (CF)
- Content-Based Filtering (CB)
- Hybrid System

## Metrics
- RMSE (Root Mean Squared Error)
- MAE (Mean Absolute Error)


In [1]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings('ignore')

# Thiết lập hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Đã import các thư viện thành công!")


Đã import các thư viện thành công!


## 1. Load Models và Dữ liệu


In [2]:
# Load models và dữ liệu
print("=" * 60)
print("ĐANG LOAD MODELS VÀ DỮ LIỆU...")
print("=" * 60)

# Load SVD model
with open('../models/svd_model.pkl', 'rb') as f:
    svd = pickle.load(f)

# Load TF-IDF vectorizer
with open('../models/tfidf_vectorizer.pkl', 'rb') as f:
    tfidf = pickle.load(f)

# Load factors
user_factors = np.load('../models/user_factors.npy')
item_factors = np.load('../models/item_factors.npy')

# Load mappings
with open('../models/movie_id_to_idx.pkl', 'rb') as f:
    movie_id_to_idx = pickle.load(f)

with open('../models/user_id_to_idx.pkl', 'rb') as f:
    user_id_to_idx = pickle.load(f)

with open('../models/tfidf_movie_id_to_row.pkl', 'rb') as f:
    tfidf_movie_id_to_row = pickle.load(f)

# Load dataframes
movies_df_clean = pd.read_pickle('../models/movies_df_clean.pkl')
train_df = pd.read_pickle('../models/train_df.pkl')
tfidf_df = pd.read_pickle('../models/tfidf_df.pkl')

# Tạo TF-IDF matrix từ tfidf_df
tfidf_matrix = csr_matrix(tfidf_df.values)

print(f"\n✓ Đã load thành công!")
print(f"  - User factors shape: {user_factors.shape}")
print(f"  - Item factors shape: {item_factors.shape}")
print(f"  - TF-IDF matrix shape: {tfidf_matrix.shape}")
print(f"  - Train ratings: {len(train_df):,}")


ĐANG LOAD MODELS VÀ DỮ LIỆU...

✓ Đã load thành công!
  - User factors shape: (668, 50)
  - Item factors shape: (3854, 50)
  - TF-IDF matrix shape: (3855, 21)
  - Train ratings: 75,296


## 2. Tạo Test Set


In [3]:
# Load dữ liệu gốc và tạo test set (giống như trong train_models.py)
print("=" * 60)
print("TẠO TEST SET...")
print("=" * 60)

# Load dữ liệu gốc
ratings_df = pd.read_csv('../data/ratings.csv')

# Làm sạch dữ liệu (giống train_models.py)
ratings_df_clean = ratings_df.dropna(subset=['userId', 'movieId', 'rating'])
ratings_df_clean = ratings_df_clean.drop_duplicates(subset=['userId', 'movieId'], keep='last')
ratings_df_clean = ratings_df_clean[
    (ratings_df_clean['rating'] >= 0.5) & 
    (ratings_df_clean['rating'] <= 5.0)
]

# Chỉ giữ các phim có trong movies_df_clean
ratings_df_clean = ratings_df_clean[ratings_df_clean['movieId'].isin(movies_df_clean['movieId'])]

# Chia train/test với cùng random_state=42
train_df_new, test_df = train_test_split(
    ratings_df_clean,
    test_size=0.2,
    random_state=42
)

print(f"Train set: {len(train_df_new):,} ratings")
print(f"Test set: {len(test_df):,} ratings")
print(f"\n✓ Đã tạo test set thành công!")


TẠO TEST SET...
Train set: 75,296 ratings
Test set: 18,825 ratings

✓ Đã tạo test set thành công!


## 3. Định nghĩa các hàm dự đoán


In [4]:
# Hàm dự đoán Collaborative Filtering
def predict_rating_cf(user_id, movie_id, user_factors, item_factors, user_id_to_idx, movie_id_to_idx, train_df):
    """Dự đoán rating sử dụng Collaborative Filtering"""
    if user_id not in user_id_to_idx or movie_id not in movie_id_to_idx:
        return train_df['rating'].mean()
    
    user_idx = user_id_to_idx[user_id]
    movie_idx = movie_id_to_idx[movie_id]
    
    prediction = np.dot(user_factors[user_idx], item_factors[movie_idx])
    prediction = np.clip(prediction, 0.5, 5.0)
    return prediction

# Hàm dự đoán Content-Based Filtering
def predict_rating_cb(user_id, movie_id, train_df, tfidf_matrix, tfidf_movie_id_to_row):
    """Dự đoán rating sử dụng Content-Based Filtering"""
    user_ratings = train_df.loc[train_df['userId'] == user_id, ['movieId', 'rating']]
    if user_ratings.empty:
        return float(train_df['rating'].mean())

    target_row = tfidf_movie_id_to_row.get(int(movie_id))
    if target_row is None:
        return float(train_df['rating'].mean())

    rated = user_ratings.copy()
    rated['row'] = rated['movieId'].map(tfidf_movie_id_to_row)
    rated = rated.dropna(subset=['row'])
    if rated.empty:
        return float(train_df['rating'].mean())

    rated_rows = rated['row'].astype(int).to_numpy()
    ratings = rated['rating'].to_numpy(dtype=float)

    # Tính similarity
    sims = (tfidf_matrix[rated_rows] @ tfidf_matrix[target_row].T).toarray().ravel()

    similarity_sum = np.abs(sims).sum()
    if similarity_sum == 0:
        return float(train_df['rating'].mean())

    pred = float((sims * ratings).sum() / similarity_sum)
    pred = float(np.clip(pred, 0.5, 5.0))
    return pred

# Hàm dự đoán Hybrid
def predict_rating_hybrid(user_id, movie_id, 
                         user_factors, item_factors, user_id_to_idx, movie_id_to_idx,
                         train_df, tfidf_matrix, tfidf_movie_id_to_row,
                         cf_weight=0.6, cb_weight=0.4):
    """Dự đoán rating sử dụng Hybrid approach"""
    cf_pred = predict_rating_cf(
        user_id, movie_id,
        user_factors, item_factors,
        user_id_to_idx, movie_id_to_idx,
        train_df
    )

    cb_pred = predict_rating_cb(
        user_id, movie_id,
        train_df, tfidf_matrix, tfidf_movie_id_to_row
    )

    hybrid_pred = cf_weight * cf_pred + cb_weight * cb_pred
    return hybrid_pred, cf_pred, cb_pred

print("✓ Đã định nghĩa các hàm dự đoán!")


✓ Đã định nghĩa các hàm dự đoán!


## 4. Đánh giá trên Test Set


In [ ]:
# Đánh giá trên test set
print("=" * 60)
print("ĐANG ĐÁNH GIÁ TRÊN TEST SET...")
print("=" * 60)
print(f"Số lượng samples trong test set: {len(test_df):,}")
print("Đang tính toán predictions...")

# Lấy sample để test nhanh (có thể comment để chạy full test set)
# test_df_sample = test_df.sample(n=min(1000, len(test_df)), random_state=42)
test_df_sample = test_df  # Chạy full test set

# Dự đoán cho từng phương pháp
predictions_cf = []
predictions_cb = []
predictions_hybrid = []
actual_ratings = []

for idx, row in test_df_sample.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']
    actual_rating = row['rating']
    
    # CF prediction
    cf_pred = predict_rating_cf(
        user_id, movie_id,
        user_factors, item_factors,
        user_id_to_idx, movie_id_to_idx,
        train_df
    )
    
    # CB prediction
    cb_pred = predict_rating_cb(
        user_id, movie_id,
        train_df, tfidf_matrix, tfidf_movie_id_to_row
    )
    
    # Hybrid prediction
    hybrid_pred, _, _ = predict_rating_hybrid(
        user_id, movie_id,
        user_factors, item_factors, user_id_to_idx, movie_id_to_idx,
        train_df, tfidf_matrix, tfidf_movie_id_to_row
    )
    
    predictions_cf.append(cf_pred)
    predictions_cb.append(cb_pred)
    predictions_hybrid.append(hybrid_pred)
    actual_ratings.append(actual_rating)
    
    # Progress indicator
    if (len(predictions_cf) % 1000 == 0):
        print(f"  Đã xử lý: {len(predictions_cf):,} / {len(test_df_sample):,} samples")

print(f"\n✓ Đã hoàn thành dự đoán cho {len(predictions_cf):,} samples!")


ĐANG ĐÁNH GIÁ TRÊN TEST SET...
Số lượng samples trong test set: 18,825
Đang tính toán predictions...
  Đã xử lý: 1,000 / 18,825 samples
  Đã xử lý: 2,000 / 18,825 samples
  Đã xử lý: 3,000 / 18,825 samples
  Đã xử lý: 4,000 / 18,825 samples
  Đã xử lý: 5,000 / 18,825 samples
  Đã xử lý: 6,000 / 18,825 samples
  Đã xử lý: 7,000 / 18,825 samples
  Đã xử lý: 8,000 / 18,825 samples
  Đã xử lý: 9,000 / 18,825 samples
  Đã xử lý: 10,000 / 18,825 samples
  Đã xử lý: 11,000 / 18,825 samples
  Đã xử lý: 12,000 / 18,825 samples
  Đã xử lý: 13,000 / 18,825 samples


## 5. Tính RMSE và MAE


In [ ]:
# Chuyển đổi sang numpy array
actual_ratings = np.array(actual_ratings)
predictions_cf = np.array(predictions_cf)
predictions_cb = np.array(predictions_cb)
predictions_hybrid = np.array(predictions_hybrid)

# Tính RMSE và MAE cho từng phương pháp
print("=" * 60)
print("KẾT QUẢ ĐÁNH GIÁ")
print("=" * 60)

# Collaborative Filtering
rmse_cf = np.sqrt(mean_squared_error(actual_ratings, predictions_cf))
mae_cf = mean_absolute_error(actual_ratings, predictions_cf)

# Content-Based Filtering
rmse_cb = np.sqrt(mean_squared_error(actual_ratings, predictions_cb))
mae_cb = mean_absolute_error(actual_ratings, predictions_cb)

# Hybrid System
rmse_hybrid = np.sqrt(mean_squared_error(actual_ratings, predictions_hybrid))
mae_hybrid = mean_absolute_error(actual_ratings, predictions_hybrid)

# Hiển thị kết quả
results_df = pd.DataFrame({
    'Phương pháp': ['Collaborative Filtering', 'Content-Based Filtering', 'Hybrid System'],
    'RMSE': [rmse_cf, rmse_cb, rmse_hybrid],
    'MAE': [mae_cf, mae_cb, mae_hybrid]
})

print("\n" + results_df.to_string(index=False))
print("\n" + "=" * 60)


## 6. So sánh chi tiết


In [ ]:
# So sánh chi tiết
print("\n" + "=" * 60)
print("SO SÁNH CHI TIẾT")
print("=" * 60)

print(f"\n📊 Collaborative Filtering:")
print(f"   RMSE: {rmse_cf:.4f}")
print(f"   MAE:  {mae_cf:.4f}")
print(f"   Improvement vs Baseline (mean): {((actual_ratings.mean() - rmse_cf) / actual_ratings.mean() * 100):.2f}%")

print(f"\n📊 Content-Based Filtering:")
print(f"   RMSE: {rmse_cb:.4f}")
print(f"   MAE:  {mae_cb:.4f}")
print(f"   Improvement vs Baseline (mean): {((actual_ratings.mean() - rmse_cb) / actual_ratings.mean() * 100):.2f}%")

print(f"\n📊 Hybrid System (CF: 0.6, CB: 0.4):")
print(f"   RMSE: {rmse_hybrid:.4f}")
print(f"   MAE:  {mae_hybrid:.4f}")
print(f"   Improvement vs CF: {((rmse_cf - rmse_hybrid) / rmse_cf * 100):.2f}%")
print(f"   Improvement vs CB: {((rmse_cb - rmse_hybrid) / rmse_cb * 100):.2f}%")

# Tìm phương pháp tốt nhất
best_method = results_df.loc[results_df['RMSE'].idxmin(), 'Phương pháp']
print(f"\n🏆 Phương pháp tốt nhất (RMSE thấp nhất): {best_method}")

# Lưu kết quả
results_df.to_csv('../results/rmse_mae_results.csv', index=False)
print(f"\n✓ Đã lưu kết quả vào: ../results/rmse_mae_results.csv")


## 7. Visualization


In [ ]:
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Tạo figure với 2 subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Subplot 1: Bar chart so sánh RMSE và MAE
x = np.arange(len(results_df))
width = 0.35

axes[0].bar(x - width/2, results_df['RMSE'], width, label='RMSE', alpha=0.8)
axes[0].bar(x + width/2, results_df['MAE'], width, label='MAE', alpha=0.8)
axes[0].set_xlabel('Phương pháp')
axes[0].set_ylabel('Error')
axes[0].set_title('So sánh RMSE và MAE')
axes[0].set_xticks(x)
axes[0].set_xticklabels(results_df['Phương pháp'], rotation=15, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Subplot 2: Scatter plot Actual vs Predicted (Hybrid)
axes[1].scatter(actual_ratings, predictions_hybrid, alpha=0.3, s=10)
axes[1].plot([actual_ratings.min(), actual_ratings.max()], 
             [actual_ratings.min(), actual_ratings.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual Rating')
axes[1].set_ylabel('Predicted Rating (Hybrid)')
axes[1].set_title('Actual vs Predicted Ratings (Hybrid System)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Đã tạo visualization!")


## 8. Phân tích lỗi


In [ ]:
# Phân tích lỗi chi tiết
print("=" * 60)
print("PHÂN TÍCH LỖI CHI TIẾT")
print("=" * 60)

# Tính errors
errors_cf = actual_ratings - predictions_cf
errors_cb = actual_ratings - predictions_cb
errors_hybrid = actual_ratings - predictions_hybrid

# Thống kê lỗi
error_stats = pd.DataFrame({
    'Phương pháp': ['Collaborative Filtering', 'Content-Based Filtering', 'Hybrid System'],
    'Mean Error': [errors_cf.mean(), errors_cb.mean(), errors_hybrid.mean()],
    'Std Error': [errors_cf.std(), errors_cb.std(), errors_hybrid.std()],
    'Min Error': [errors_cf.min(), errors_cb.min(), errors_hybrid.min()],
    'Max Error': [errors_cf.max(), errors_cb.max(), errors_hybrid.max()],
    'RMSE': [rmse_cf, rmse_cb, rmse_hybrid],
    'MAE': [mae_cf, mae_cb, mae_hybrid]
})

print("\nThống kê lỗi:")
print(error_stats.to_string(index=False))

# Phân tích theo khoảng rating
print("\n" + "=" * 60)
print("PHÂN TÍCH LỖI THEO KHOẢNG RATING")
print("=" * 60)

# Chia thành các khoảng rating
bins = [0.5, 2.0, 3.5, 5.0]
labels = ['Low (0.5-2.0)', 'Medium (2.0-3.5)', 'High (3.5-5.0)']
rating_bins = pd.cut(actual_ratings, bins=bins, labels=labels)

# Tính RMSE và MAE cho từng khoảng
for label in labels:
    mask = rating_bins == label
    if mask.sum() > 0:
        actual_bin = actual_ratings[mask]
        pred_cf_bin = predictions_cf[mask]
        pred_cb_bin = predictions_cb[mask]
        pred_hybrid_bin = predictions_hybrid[mask]
        
        rmse_cf_bin = np.sqrt(mean_squared_error(actual_bin, pred_cf_bin))
        mae_cf_bin = mean_absolute_error(actual_bin, pred_cf_bin)
        rmse_cb_bin = np.sqrt(mean_squared_error(actual_bin, pred_cb_bin))
        mae_cb_bin = mean_absolute_error(actual_bin, pred_cb_bin)
        rmse_hybrid_bin = np.sqrt(mean_squared_error(actual_bin, pred_hybrid_bin))
        mae_hybrid_bin = mean_absolute_error(actual_bin, pred_hybrid_bin)
        
        print(f"\n{label} (n={mask.sum():,}):")
        print(f"  CF:     RMSE={rmse_cf_bin:.4f}, MAE={mae_cf_bin:.4f}")
        print(f"  CB:     RMSE={rmse_cb_bin:.4f}, MAE={mae_cb_bin:.4f}")
        print(f"  Hybrid: RMSE={rmse_hybrid_bin:.4f}, MAE={mae_hybrid_bin:.4f}")

print("\n" + "=" * 60)
print("HOÀN THÀNH ĐÁNH GIÁ!")
print("=" * 60)
